In [68]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import talib as tb
import yfinance as yf

import pickle


import datetime
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

In [69]:
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F',  
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

In [70]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )


In [71]:
df_coin

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-05-20,30311.119141,30664.976562,28793.605469,29200.740234,29200.740234,30749382605
2022-05-21,29199.859375,29588.869141,29027.394531,29432.226562,29432.226562,17274840442
2022-05-22,29432.472656,30425.861328,29275.183594,30323.722656,30323.722656,21631532270


In [72]:
df_coin = df_converter(df_coin)

In [73]:
df_coin.index

DatetimeIndex(['2017-11-09', '2017-11-10', '2017-11-11', '2017-11-12',
               '2017-11-13', '2017-11-14', '2017-11-15', '2017-11-16',
               '2017-11-17', '2017-11-18',
               ...
               '2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19',
               '2022-05-20', '2022-05-21', '2022-05-22', '2022-05-23',
               '2022-05-24', '2022-05-25'],
              dtype='datetime64[ns]', name='Date', length=1659, freq=None)

In [74]:
start_date = '2021-10-22'

In [75]:
start_date = pd.to_datetime(start_date)

In [76]:
from datetime import datetime


In [77]:
start_date

Timestamp('2021-10-22 00:00:00')

In [78]:
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-21,29199.859375,29588.869141,29027.394531,29432.226562,29432.226562,1.727484e+10,30500.734375,12232.171066,31154.594764,42.679227,...,34198.881372,-2512.732215,-2583.118995,70.386780,36.324105,1849.073499,39.043883,36.340600,103.172997,4123.339844
2022-05-22,29432.472656,30425.861328,29275.183594,30323.722656,30323.722656,2.163153e+10,30155.519010,12238.620817,31050.735751,42.218628,...,32712.477177,-2348.220816,-2536.139359,187.918543,39.872756,1278.479083,45.973946,37.380500,103.172997,4123.339844
2022-05-23,30309.396484,30590.585938,28975.560547,29098.910156,29098.910156,3.148345e+10,29824.828646,12244.629473,30806.757551,41.917658,...,31352.744383,-2290.275608,-2486.966609,196.691001,36.849768,763.957869,46.305612,43.774481,102.097000,4123.339844


In [79]:
df_coin = df_coin[df_coin.index >= start_date]

In [80]:
df_coin.head()

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-22,62237.890625,63715.023438,60122.796875,60692.265625,60692.265625,3.843408e+10,59411.947135,9548.177219,59279.606622,42.268428,...,66512.766057,4029.816080,3784.073284,245.742796,61.311059,3550.409461,45.656328,70.563090,93.625000,4544.899902
2021-10-23,60694.628906,61743.878906,59826.523438,61393.617188,61393.617188,2.688255e+10,59906.998437,9568.163896,59543.857943,41.723052,...,66432.717203,3820.417900,3791.342207,29.075693,62.480777,3262.859383,23.700462,48.104014,93.625000,4544.899902
2021-10-24,61368.343750,61505.804688,59643.343750,60930.835938,60930.835938,2.731618e+10,60304.505990,9587.956833,59717.230192,41.121085,...,66281.803561,3575.904948,3748.254755,-172.349808,61.173225,2988.648786,16.516932,28.624574,93.625000,4544.899902
2021-10-25,60893.925781,63729.324219,60691.800781,63039.824219,63039.824219,3.106491e+10,60855.722396,9608.546920,60132.554446,41.093764,...,66179.665938,3511.822299,3700.968264,-189.145965,64.772807,2661.971771,29.515861,23.244418,93.808998,4566.479980
2021-10-26,63032.761719,63229.027344,59991.160156,60363.792969,60363.792969,3.487897e+10,61047.655990,9628.090719,60161.459261,40.705583,...,66070.739534,3208.121750,3602.398961,-394.277211,57.522762,2511.541772,28.040026,24.690939,93.941002,4574.790039


# Predictions Graph

In [81]:
def add_days(df, forecast_length):
    end_point = len(df)
    df1 = pd.DataFrame(index=range(forecast_length), columns=range(2))
    df1.columns = ['Close', 'Date']
    df = df.append(df1)
    df = df.reset_index(drop=True)
    x = df.at[end_point - 1, 'Date']
    x = pd.to_datetime(x, format='%Y-%m-%d')
    for i in range(forecast_length):
        df.at[df.index[end_point + i], 'Date'] = x + timedelta(days=1+i)
        df.at[df.index[end_point + i], 'Close'] = 0
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.drop(['Date'], axis=1)
    return df

In [82]:
def forecasting(model,df1, forecast_length):
    df3 = df1[['Close', 'Date']]
    df3 = add_days(df3, forecast_length)
    finaldf = df1.drop('Date', axis=1)
    finaldf = finaldf.reset_index(drop=True)
    end_point = len(finaldf)
    x = end_point - forecast_length
    finaldf_train = finaldf.loc[:x - 1, :]
    finaldf_train_x = finaldf_train.loc[:, finaldf_train.columns != 'Close']
    finaldf_train_y = finaldf_train['Close']

    fit = model.fit(finaldf_train_x, finaldf_train_y)
    yhat = []
    end_point = len(finaldf)
    df3_end = len(df3)
    for i in range(forecast_length, 0, -1):
        y = end_point - i
        inputfile = finaldf.loc[y:end_point, :]
        inputfile_x = inputfile.loc[:, inputfile.columns != 'Close']
        pred_set = inputfile_x.head(1)
        pred = fit.predict(pred_set)
        df3.at[df3.index[df3_end - i], 'Close'] = pred[0]
        finaldf = df1.drop('Date', axis=1)
        finaldf = finaldf.reset_index(drop=True)
        yhat.append(pred)
    yhat = np.array(yhat)
    return yhat

In [83]:
def predictions(df_coin,model, forecast_lenght = 5, train_lenght = 100,target = 'Close'):
    """ df_coin must be with date in index,
        forecast_lenght is the amount of days that we will predict
        model is the model predefined to use to get our predictions
        train_length is the amount of days that we will use to train the model
        target is what we are predicting
        This will return a graphic that will contain the data from the train set and our predictions
    """
    df_coin = df_coin.tail(train_lenght)
    df_coin.reset_index(inplace=True)
    df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=df_coin['Date'], y=df_coin[target], 
                    name='Actual Values', mode='lines',line=dict(color='black')))
    #Predictions
    forecast = forecasting(model,df_coin,forecast_lenght)
    #df that will contain the predictions
    df_pred = pd.DataFrame(columns=['Date',target])
    #Adding the predictions to our dataset
    for day, x in enumerate(forecast):
        new_row={'Date':df_coin['Date'].max() + timedelta(days=1+day),
     target:x[0]}
        df_pred = df_pred.append(new_row, ignore_index=True)

    df_pred['Date'] = pd.to_datetime(df_pred['Date'], format='%Y-%m-%d')

    fig2.add_trace(go.Scatter(x=df_pred['Date'], y=df_pred[target], name='Predictions', mode='lines',line=dict(color='red')))
    fig2.update_layout(dict(updatemenus=[
                        dict(
                        type = "buttons",
                        direction = "left",
                        buttons=list([
                                dict(
                                args=["visible", "legendonly"],
                                label="Deselect All",
                                method="restyle"
                                ),
                                dict(
                                args=["visible", True],
                                label="Select All",
                                method="restyle"
                                )
                        ]),
                        pad={"r": 10, "t": 10},
                        showactive=False,
                        x=1,
                        xanchor="right",
                        y=1.1,
                        yanchor="top"
                        ),
                ]
        ))
    fig2.show()

In [84]:
from sklearn.ensemble import RandomForestRegressor
rf_final = RandomForestRegressor(random_state=10,criterion='mae', max_depth=20, max_features='auto', n_estimators=30)

In [85]:
predictions(df_coin, rf_final, 5, 20, 'Close')

C:\Users\Admin\AppData\Local\Temp/ipykernel_17036/2344453812.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\ensemble\_forest.py:395: FutureWarning:

Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.



In [86]:
import finance_lib as fb

In [87]:
fb.predictions(df_coin, rf_final, 5, 100, 'Close')

c:\Users\Admin\Desktop\NOVAIMS\BC\BC5\finance_lib.py:149: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\ensemble\_forest.py:395: FutureWarning:

Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.



## Market Info

### Days Variation

In [257]:
def get_percentage_img(current_value, prev_value, height_size):
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode = "number+delta",
        title = {"text": "1D Price Change"},
        value = current_value,
        delta = {'reference': prev_value, 'relative': True, 'valueformat':'.3%'},
        number={'valueformat':".5f",'prefix':'$'},
        domain = {'x': [0, 1], 'y': [0, 1]}))
    fig.update_layout(
            template="plotly_dark",
            plot_bgcolor = 'rgba(0, 0, 0, 0)',
            paper_bgcolor = 'rgba(0, 0, 0, 0)',
            font_color="white",
            font_size= 500,
            margin={'t': 0,'l':0,'b':10,'r':0},
            height=height_size
        )

    return fig

### Open Close Volume

In [258]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )
df_coin_day = yf.download('BTC-USD',
                      start=date.today(), 
                      interval="1m",
                      progress=False,
    )

In [259]:
# crypto first day in dataset
crypto_first_day = str(df_coin.index.min())
crypto_first_day = crypto_first_day[:10]
crypto_first_day
date_time_obj = datetime.strptime(crypto_first_day, '%Y-%m-%d')
date_time_obj

datetime.datetime(2014, 9, 17, 0, 0)

In [260]:
 # first viz 
today = pd.to_datetime(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=0))
df_coin_day.index = df_coin_day.index.tz_localize(None)
current_price = df_coin_day['Close'].iloc[-1]
today_open_price_not_round = df_coin_day[df_coin_day.index >= today]['Open'][0]    
curr_price_fig = get_percentage_img(current_price, today_open_price_not_round, 80)
curr_price_fig

In [236]:
"${0:10.5f}".format(current_price)

'$29547.85742'

In [183]:
df_coin_day[df_coin_day.index >= today]['Open'][0]

0.9990105628967285

In [184]:
current_price

0.9989119172096252

In [185]:
df_coin_day.index = df_coin_day.index.tz_localize(None)
today = pd.to_datetime(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=0))
today_open_price = round(df_coin_day[df_coin_day.index >= today]['Open'][0],5)
today_open_price

0.99901

In [186]:
fiftytwo_weeks = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(weeks=52))
high_fiftytwo_weeks=df_coin[df_coin.index >= fiftytwo_weeks]['High'].max()
low_fiftytwo_weeks=df_coin[df_coin.index >= fiftytwo_weeks]['Low'].min()
price_range_weeks = str(round(high_fiftytwo_weeks, 2))+'-'+str(round(low_fiftytwo_weeks, 2))
price_range_weeks

'1.03-0.95'

In [188]:
# last volume 
yesterday = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=1))
weekly_yesterday = pd.to_datetime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)-timedelta(days=7))
before_volume = df_coin[df_coin.index == weekly_yesterday]['Volume'][0]
last_volume = df_coin[df_coin.index == yesterday]['Volume'][0]
get_percentage_img(last_volume, before_volume,50)

In [189]:
today_high = df_coin_day[df_coin_day.index >= today]['High'].max()
today_low = df_coin_day[df_coin_day.index >= today]['Low'].min()
price_range = str(round(today_high, 5))+'-'+str(round(today_low, 5))
price_range

'0.99918-0.99882'

In [190]:
import requests

url = 'https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
list_crypto=[]
for start in range(1, 20000, 5000):

    params = {
        'start': start,
        'limit': 5000,
    }

    r = requests.get(url, params=params)
    data = r.json()
    for number, item in enumerate(data['data']):
        a = f"{item['symbol']}-USD"
        list_crypto.append(a)

In [ ]:
list_crypto

['BTC-USD',
 'ETH-USD',
 'USDT-USD',
 'BNB-USD',
 'USDC-USD',
 'XRP-USD',
 'BUSD-USD',
 'ADA-USD',
 'SOL-USD',
 'DOGE-USD',
 'DOT-USD',
 'WBTC-USD',
 'AVAX-USD',
 'TRX-USD',
 'DAI-USD',
 'SHIB-USD',
 'MATIC-USD',
 'LTC-USD',
 'CRO-USD',
 'LEO-USD',
 'NEAR-USD',
 'FTT-USD',
 'UNI-USD',
 'BCH-USD',
 'XMR-USD',
 'LINK-USD',
 'XLM-USD',
 'ETC-USD',
 'ATOM-USD',
 'FLOW-USD',
 'ALGO-USD',
 'APE-USD',
 'HBAR-USD',
 'VET-USD',
 'EGLD-USD',
 'MANA-USD',
 'XTZ-USD',
 'ICP-USD',
 'FIL-USD',
 'SAND-USD',
 'KCS-USD',
 'AAVE-USD',
 'ZEC-USD',
 'EOS-USD',
 'KLAY-USD',
 'AXS-USD',
 'THETA-USD',
 'MKR-USD',
 'TUSD-USD',
 'HT-USD',
 'FTM-USD',
 'GRT-USD',
 'BTT-USD',
 'BSV-USD',
 'RUNE-USD',
 'XEC-USD',
 'USDP-USD',
 'MIOTA-USD',
 'HNT-USD',
 'CAKE-USD',
 'GMT-USD',
 'USDN-USD',
 'QNT-USD',
 'NEO-USD',
 'STX-USD',
 'UST-USD',
 'OKB-USD',
 'LRC-USD',
 'NEXO-USD',
 'CHZ-USD',
 'KSM-USD',
 'ZIL-USD',
 'DASH-USD',
 'CVX-USD',
 'PAXG-USD',
 'WAVES-USD',
 'ENJ-USD',
 'BAT-USD',
 'CELO-USD',
 'GALA-USD',
 'CRV

# Graph para client evolution

In [ ]:
df_time = pd.read_csv (r'./client_valuev2_test.csv')

In [ ]:
df_time = df_time.copy()

In [ ]:
fig = px.line(x=df_time.time, y=df_time.current_value)
fig.add_trace(
    go.Scatter(
        x=df_time.time,
        y=df_time.spent,
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.show()

In [ ]:
df_coin_day = yf.download('BTC-USD',
                      start=date.today(), 
                      interval="1m",
                      progress=False,
    )

In [ ]:
df_coin_day

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-05-24 23:00:00+00:00,29583.810547,29583.810547,29583.810547,29583.810547,29583.810547,0
2022-05-24 23:01:00+00:00,29596.119141,29596.119141,29596.119141,29596.119141,29596.119141,0
2022-05-24 23:02:00+00:00,29595.556641,29595.556641,29595.556641,29595.556641,29595.556641,0
2022-05-24 23:03:00+00:00,29592.214844,29592.214844,29592.214844,29592.214844,29592.214844,4296704
2022-05-24 23:04:00+00:00,29582.710938,29582.710938,29582.710938,29582.710938,29582.710938,0
...,...,...,...,...,...,...
2022-05-25 12:16:00+00:00,29486.515625,29486.515625,29486.515625,29486.515625,29486.515625,894976
2022-05-25 12:17:00+00:00,29489.929688,29489.929688,29489.929688,29489.929688,29489.929688,4687872
2022-05-25 12:18:00+00:00,29475.775391,29475.775391,29475.775391,29475.775391,29475.775391,0


In [265]:
!pip uninstall xgboost -y

Found existing installation: xgboost 1.6.0
Uninstalling xgboost-1.6.0:
  Successfully uninstalled xgboost-1.6.0


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_internal\cli\base_command.py", line 173, in _main
    status = self.run(options, args)
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_internal\commands\uninstall.py", line 97, in run
    uninstall_pathset.commit()
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_internal\req\req_uninstall.py", line 436, in commit
    self._moved_paths.commit()
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_internal\req\req_uninstall.py", line 287, in commit
    save_dir.cleanup()
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_internal\utils\temp_dir.py", line 184, in cleanup
    rmtree(self._path)
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 326, in wrapped_f
    return self(f, *args, **kw)
  File "C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\pip\_vendor\tenacity\